# Week 4 Exercise

## Goal
Create a script to translate Hirigana (ひりがな）to Katakana (カタカナ）to Kanji (漢字) using a specific LLM model (closed or open source).


In [64]:
import os
import io
import sys
import json
import requests
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr
import subprocess

from sympy.strategies.core import switch

**Domain-Level Exceptions**

In [92]:
class EnvironmentException(Exception):
    def __init__(self, message: str, cause: Exception | None = None):
        self._message = message
        self._cause = cause

    @property
    def message(self) -> str:
        return self._message

    @property
    def cause(self) -> Exception | None:
        return self._cause

class ApiClientException(Exception):
    """Raise this exception upon errors with API client wrappers."""
    def __init__(self, message: str, cause: Exception | None = None):
        self._message = message
        self._cause = cause

    @property
    def message(self) -> str:
        return self._message

    @property
    def cause(self) -> Exception | None:
        return self._cause

**Application-level configurations**

In [66]:
DEFAULT_OPENAI_MODEL = "gpt-4o"
DEFAULT_CLAUDE_MODEL = "claude-3-5-sonnet-20240620"
# Toggle gradio auto-launching the UI.
DEFAULT_GRADIO_UI_AUTO_LAUNCH = True

_CONFIG = {
    'OPENAI_MODEL': DEFAULT_OPENAI_MODEL,
    'CLAUDE_MODEL': DEFAULT_CLAUDE_MODEL,
    'GRADIO_UI_AUTO_LAUNCH': DEFAULT_GRADIO_UI_AUTO_LAUNCH
}


**Load API Keys**

In [67]:
# Load API Keys.
try:
    load_dotenv(override=True)
    os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
    os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
    os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')
except Exception as e:
    error_message = "Failure to setup environment variables, please check your configuration."
    print(error_message)
    raise EnvironmentException(message=error_message, cause=e)
print("API Keys loaded!")

API Keys loaded!


**Defining System Prompt**

In [40]:
_SYSTEM_PROMPT = "You are an assistant that translates Japanese hirigana (ひりがな) to its closest kanji (漢字). "
_SYSTEM_PROMPT = "The input hirigana might be in romaji or actual hirigana characters. "
_SYSTEM_PROMPT += "If you find multiple matches for the input hirigana, use your best reasoning skills to match the entire word or phrase. "
_SYSTEM_PROMPT += "If you're unable to find any match for the input hirigana, let the user know."

**Setup System Model Interfaces**

In [137]:
from enum import StrEnum

class LanguageModel(StrEnum):
    GPT = "gpt"
    CLAUDE = "claude"

class RoleName(StrEnum):
    SYSTEM = "system"
    USER = "user"

class OpenAiApiClient:
    def __init__(
        self,
        system_prompt: str,
        # todo: model version should derive from application-level config.
        model_version: str = DEFAULT_OPENAI_MODEL,
        user_prompt: str | None = None,
    ) -> None:
        """Create an OpenAI API client.

        :param system_prompt: The prompt to use for prompting for the system prompt.
                              This can be set with a later setter method.
        :param user_prompt: Optional user prompt to initialize with.
        """
        self._client = OpenAI()
        # Setup a basic message hash for the model.
        self._message_hashes = [
            {"role": RoleName.SYSTEM, "content": system_prompt},
            {"role": RoleName.USER, "content": user_prompt},
        ]
        self._model_version = model_version

    def set_system_prompt(self, system_prompt: str) -> None:
        """Set the system prompt to a new value."""
        print(f"Setting SystemPrompt to {system_prompt}")
        # Update internal message hash.
        self._update_message_hash(
            role=RoleName.SYSTEM, message=system_prompt
        )

    def set_user_prompt(self, user_prompt: str) -> None:
        """Set the user prompt to a new value."""
        print(f"Setting UserPrompt to {user_prompt}")
        self._update_message_hash(
            role=RoleName.USER, message=user_prompt
        )


    def chat_stream(self):
        stream = self._execute_chat_stream()
        try:
            return self.__chunk_stream(stream)
        except Exception as ex:
            err_message = "Failed to parse GPT chat stream."
            print(err_message)
            raise ApiClientException(message=err_message, cause=ex) from ex

    def _execute_chat_stream(self):
        """Execute the chat stream via GPT client.

        :return: The result of the chat stream.
        """
        try:
            print(
                f"Executing chat stream with "
                f"model {self._model_version}, "
                f" messages: {self._message_hashes}"
            )
            stream = self._client.chat.completions.create(
                model=self._model_version,
                messages=self._message_hashes,
                stream=True
            )
            return stream
        except Exception as ex:
            err_message = f"Failed to execute GPT chat stream. Error: {ex}"
            print(err_message)
            raise ApiClientException(message=err_message, cause=ex) from ex

    def _prepare_reply(self, stream):
        return self.__chunk_stream(stream)

    def __chunk_stream(
        self,
        stream,
        initial_response:str=""
    ):
        """
        Return the stream as chunks.

        :param stream: The GPT stream to chunk.
        :param initial_response: The initial string or reply to start the chunk response.
        """
        for chunk in stream:
            fragment = chunk.choices[0].delta.content or ""
            initial_response += fragment
            print(fragment, end='', flush=True)
        return initial_response

    def _update_message_hash(
        self,
        role: RoleName,
        message: str
    ) -> None:
        """Create a new message hash for the role or update the existing one."""
        # Remove the old hash and create the new one
        self.__remove_message_hash(
            role=RoleName.USER, message_hashes=self._message_hashes
        )
        self._message_hashes.append({'role': role, 'content': message})

    @classmethod
    def __remove_message_hash(
        cls,
        role: RoleName,
        message_hashes: list[dict]
    ) -> None:
        """Remove the existing message hash for a given role."""
        # Find the index of the first dictionary that contains the key.
        # next() is used with a generator expression over enumerated items.
        # If no such dictionary is found, 'None' is returned for 'index_to_remove'.
        index_to_remove = next(
            (i for i, d in enumerate(message_hashes) if role in d),
            None
        )

        if index_to_remove is not None:
            # If a dictionary is found, create a new list by concatenating
            # the part before the found index and the part after it.
            # This avoids modifying the original list and is a functional approach.
            return message_hashes[:index_to_remove] + message_hashes[index_to_remove + 1:]
        else:
            # If no dictionary with the key is found, return a copy of the original list.
            # Returning a copy is good practice in functional programming to ensure
            # the original input is never mutated.
            return list(message_hashes)




**User Prompt**

In [138]:
def user_prompt_for(hirigana: str) -> str:
    """
    Return the user prompt for a given hirigana input.

    param: hirigana: The input hirigana to translate to Kanji.
    """
    if not hirigana:
        raise ValueError("hirigana value is required!")
    _user_prompt = f"Translate the following hirigana to kanji: {hirigana}"
    return _user_prompt


In [139]:
def execute_gpt(hirigana: str):
    # Open GPT connection with system prompt.
    gpt_client = OpenAiApiClient(system_prompt=_SYSTEM_PROMPT)
    # Set user prompt for input hirigana
    gpt_client.set_user_prompt(user_prompt_for(hirigana))
    # Execute streaming
    response = gpt_client.chat_stream()
    print(f'response: {response}')
    return response

def test_execute_gpt():
    execute_gpt(hirigana="かんじ")

test_execute_gpt()

Setting UserPrompt to Translate the following hirigana to kanji: かんじ
Executing chat stream with model gpt-4o,  messages: [{'role': <RoleName.SYSTEM: 'system'>, 'content': "The input hirigana might be in romaji or actual hirigana characters. If you find multiple matches for the input hirigana, use your best reasoning skills to match the entire word or phrase. If you're unable to find any match for the input hirigana, let the user know."}, {'role': <RoleName.USER: 'user'>, 'content': None}, {'role': <RoleName.USER: 'user'>, 'content': 'Translate the following hirigana to kanji: かんじ'}]
Failed to execute GPT chat stream. Error: Error code: 400 - {'error': {'message': "Invalid value for 'content': expected a string, got null.", 'type': 'invalid_request_error', 'param': 'messages.[1].content', 'code': None}}


ApiClientException: 

In [109]:
def translate(hirigana: str, model: str):
    response = _select_model(hirigana, language_model=model)
    for stream_so_far in response:
        yield stream_so_far

def _select_model(
        hirigana: str,
        language_model: str
):
    """
    Choose the desired language model and execute the translation.
    """
    if LanguageModel.GPT.casefold() == language_model.casefold():
        return execute_gpt(hirigana=hirigana)
    elif LanguageModel.CLAUDE.casefold() == language_model.casefold():
        raise NotImplementedError("todo")
    raise ValueError(f"Invalid model: {model}")


### UI


In [110]:
import gradio as gr

with gr.Blocks() as ui:
    # gr.Markdown('### Translate Hirigana (ひらがな) to Kanji（漢字）')
    with gr.Row():
        # Input text - Corrected variable name here
        input_hirigana = gr.Textbox(label="hirigana word or phrase", lines=10)
        output_kanji = gr.Textbox(label="kanji word or phrase", lines=10)
    with gr.Row():
        # Model selection
        model = gr.Dropdown(["GPT", "Claude"], label="Select model", value="GPT")
        # Translate Button
        translate_btn = gr.Button("Translate")
    # Translation execute - Corrected input reference
    translate_btn.click(translate, inputs=[input_hirigana, model], outputs=[output_kanji])

# Launch UI.
ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7873
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/var/folders/2l/c7fg4dt56j1gxp8ls1jx0lnr0000gn/T/ipykernel_48352/3206488785.py", line 81, in _execute_chat_stream
    stream = self._client.chat.completions.create(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/llms/lib/python3.11/site-packages/openai/_utils/_utils.py", line 287, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/llms/lib/python3.11/site-packages/openai/resources/chat/completions/completions.py", line 1087, in create
    return self._post(
           ^^^^^^^^^^^
  File "/opt/anaconda3/envs/llms/lib/python3.11/site-packages/openai/_base_client.py", line 1249, in post
    return cast(ResponseT, self.request(cast_to, opts, stream=stream, stream_cls=stream_cls))
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/llms/lib/python3.11/site-packages/openai/_base_client.py", line 1

### Deterministic Unit Tests
Unit tests for deterministic logic.

#### Prerequisites:
- `pytest`

In [144]:
import unittest
from unittest.mock import MagicMock, patch
import io
import sys

# Assume these are defined elsewhere or mock them for the test
# For a bare-bones test, we'll define simple versions.
class RoleName:
    SYSTEM = "system"
    USER = "user"
    ASSISTANT = "assistant"

DEFAULT_OPENAI_MODEL = "gpt-3.5-turbo"

# Placeholder for the OpenAI class.
# In a real application, this would be 'from openai import OpenAI'.
# We define it here so OpenAiApiClient can be defined without a NameError,
# and then we can patch it correctly.
class OpenAI:
    """A dummy OpenAI class to allow OpenAiApiClient definition."""
    def __init__(self):
        pass # No actual initialization needed for the dummy

# The class to be tested (provided by the user)
class OpenAiApiClient:
    def __init__(
        self,
        system_prompt: str,
        # todo: model version should derive from application-level config.
        model_version: str = DEFAULT_OPENAI_MODEL,
        user_prompt: str | None = None,
    ) -> None:
        """Create an OpenAI API client.

        :param system_prompt: The prompt to use for prompting for the system prompt.
                              This can be set with a later setter method.
        :param user_prompt: Optional user prompt to initialize with.
        """
        # This is the line we want to mock.
        # When the test runs, the 'OpenAI' class will be replaced by a MagicMock
        # due to the @patch decorator in the test class.
        self._client = OpenAI()
        # Setup a basic message hash for the model.
        self._message_hashes = [
            {"role": RoleName.SYSTEM, "content": system_prompt},
            {"role": RoleName.USER, "content": user_prompt},
        ]
        self._model_version = model_version

# Bare-bones unit test class
class TestOpenAiApiClient(unittest.TestCase):
    """
    A bare-bones unit test class for the OpenAiApiClient.
    This class uses unittest.mock.patch to mock the external OpenAI dependency,
    allowing for isolated testing of the OpenAiApiClient's initialization logic.
    """

    # The patch target is now 'OpenAI' in the current module (__main__ in Jupyter).
    # This will replace the 'OpenAI' class defined above with a MagicMock.
    @patch('__main__.OpenAI')
    def setUp(self, MockOpenAI):
        """
        Set up the test environment before each test method.
        This method is called automatically by the unittest framework.
        It initializes a mock for the OpenAI client and creates an instance
        of OpenAiApiClient with predefined prompts.
        """
        # MockOpenAI is the MagicMock replacing the actual OpenAI class.
        # mock_openai_instance is the result of calling MockOpenAI(),
        # which is what self._client will be assigned to in OpenAiApiClient's __init__.
        self.mock_openai_instance = MockOpenAI.return_value

        self.system_prompt = "You are a helpful assistant."
        self.user_prompt = "Hello, world!"
        self.client = OpenAiApiClient(
            system_prompt=self.system_prompt,
            user_prompt=self.user_prompt
        )

    def test_initialization(self):
        """
        Test that the OpenAiApiClient is initialized correctly.
        This test verifies:
        1. The internal _client attribute is an instance of the mocked OpenAI client.
        2. The _message_hashes list is correctly populated with system and user prompts.
        3. The _model_version is set to the default if not specified.
        """
        # Assert that the _client attribute is the mocked OpenAI instance
        # This confirms that self._client = OpenAI() inside the class
        # indeed called the mocked OpenAI and got its return value.
        self.assertEqual(self.client._client, self.mock_openai_instance)

        # Assert that _message_hashes is correctly initialized
        expected_message_hashes = [
            {"role": RoleName.SYSTEM, "content": self.system_prompt},
            {"role": RoleName.USER, "content": self.user_prompt},
        ]
        self.assertEqual(self.client._message_hashes, expected_message_hashes)

        # Assert that _model_version is set to the default
        self.assertEqual(self.client._model_version, DEFAULT_OPENAI_MODEL)

    def test_initialization_no_user_prompt(self):
        """
        Test initialization when no user prompt is provided.
        Ensures that _message_hashes handles a None user_prompt correctly.
        """
        # Create a new client instance without a user prompt
        client_no_user = OpenAiApiClient(system_prompt="Another system prompt.")

        expected_message_hashes = [
            {"role": RoleName.SYSTEM, "content": "Another system prompt."},
            {"role": RoleName.USER, "content": None}, # User prompt should be None
        ]
        self.assertEqual(client_no_user._message_hashes, expected_message_hashes)
        self.assertEqual(client_no_user._model_version, DEFAULT_OPENAI_MODEL)

    def test_initialization_custom_model_version(self):
        """
        Test initialization with a custom model version.
        Verifies that the provided model_version is correctly assigned.
        """
        custom_model = "gpt-4-turbo"
        client_custom_model = OpenAiApiClient(
            system_prompt="System for custom model.",
            model_version=custom_model
        )

        self.assertEqual(client_custom_model._model_version, custom_model)
        # Ensure other attributes are still correctly set
        expected_message_hashes = [
            {"role": RoleName.SYSTEM, "content": "System for custom model."},
            {"role": RoleName.USER, "content": None},
        ]
        self.assertEqual(client_custom_model._message_hashes, expected_message_hashes)


# This block is modified to run tests in a Jupyter Notebook cell
# It collects tests and runs them using TextTestRunner,
# which prints results without trying to exit the interpreter.
if __name__ == '__main__':
    # Create a test suite from the TestOpenAiApiClient class
    suite = unittest.TestSuite()